SQL Miscellanea
-----

In [ ]:
%load_ext sql
%sql sqlite:///world-db

We can create views in SQL. Views are not stored as tables (materialized), but they can be used inside other SQL queries as normal tables.

In [ ]:
%%sql
select * from Countrylanguage limit 5

In [ ]:
%%sql
DROP VIEW OfficialCountryLanguage;

CREATE VIEW OfficialCountryLanguage AS
SELECT C.Name AS CountryName, L.Language AS Language
FROM CountryLanguage L, Country C
WHERE L.CountryCode = C.Code
AND L.IsOfficial = 'T' ;

In [ ]:
%sql SELECT * FROM OfficialCountryLanguage LIMIT 10;

**COMPUTING THE LONGEST SEQUENCE OF CONSECUTIVE NUMBERS**


In the following, we are given a table with a single column. Our goal is to compute the length of the longest sequence of consecutive numbers.

We define the following table for our case:

In [ ]:
%%sql
DROP TABLE IF EXISTS R;
CREATE TABLE R(A integer);
INSERT INTO R Values (1),(2),(5),(9),(3),(10),(25);

To express this query, we make use of the function `row_number`. This function returns the sequential number of a row within a table. It can be used with an `ORDER BY` clause, so that we can specify a desired order in which we want to number the rows.

Let's see what happens if we use this!

In [ ]:
%%sql
SELECT A,(row_number() OVER (ORDER BY A ASC)) AS B
FROM R ;

In [ ]:
%%sql
WITH num AS
(SELECT A,(A - row_number() OVER (ORDER BY A)) AS B FROM R)
SELECT MIN(A) AS start, MAX(A) AS end, COUNT(*) AS length
FROM num
GROUP BY B
ORDER BY length DESC;

**MEDIAN**

In the following, we will show how one can compute the *median* of a table column using SQL. Recall that to compute the median of a (multi)set of *n* values, we first order the values; then, we return the middle number if *n* is odd, otherwise the average of the two middle numbers. We use the table R from above.

We can easily compute the median using `row_number`- the query below only works for an odd numer of elements.

In [ ]:
%%sql
WITH num AS
(SELECT A, (row_number() OVER (ORDER BY A)) AS B FROM R)
SELECT A
FROM num
WHERE B*2 = (SELECT COUNT(*) FROM num)+1;

Next, we will see how to compute the median without using `row_number`! We will first solve a simplified version of the median problem. Let's for now assume that:
* the number of elements is odd
* there are no duplicate values

In [ ]:
%%sql
SELECT X.A
FROM R AS X
WHERE (SELECT COUNT(*) FROM R AS X1 WHERE X.A > X1.A) 
= (SELECT COUNT(*) FROM R AS X2 WHERE X.A < X2.A);

The above solution will not work if our two assumptions do not hold (why?). Let us rewrite the query so that we can solve the general median problem.

In [ ]:
%%sql
SELECT AVG(DISTINCT X.A)
FROM R AS X
WHERE (SELECT COUNT(*) FROM R AS X1 WHERE X.A >= X1.A) >= (SELECT COUNT(*) FROM R AS X2 WHERE X.A < X2.A)
AND (SELECT COUNT(*) FROM R AS X1 WHERE X.A > X1.A) <= (SELECT COUNT(*) FROM R AS X2 WHERE X.A <= X2.A);

**Paths in Graphs**

We will next show how we can compute some queries on a graph. Here, we represent the graph a single relation with schema `Edge(source, target, distance)`.

In [ ]:
%%sql
DROP TABLE IF EXISTS Edge;
CREATE TABLE Edge (source integer, target integer, distance integer);
INSERT INTO Edge VALUES (1,2,10),(2,3,10),(3,4,20),(4,1,30),(1,3,5); 

The query below computes the number of outgoing edges for each vertex.

In [ ]:
%%sql
SELECT source, COUNT(target)
FROM Edge
GROUP BY source;

Next, we want to find all the directed paths of length 2 in the graph.

In [ ]:
%%sql
SELECT e1.source, e1.target, e2.target, e3.target
FROM Edge e1, Edge e2, Edge e3
WHERE e1.target = e2.source AND e2.target = e3.source; 

Can we also compute the distance of each path?

In [ ]:
%%sql
SELECT e1.source, e1.target, e2.target, (e1.distance+e2.distance)
FROM Edge e1, Edge e2
WHERE e1.target = e2.source; 